# (HiddenState:Path)[V1]

In [ ]:
hmm = reload(hmm)
A = hmm.get_trans_mat(match_res, roadmap)
B, D = hmm.get_obsv_mat(train, match_res)

In [ ]:
# clean test dataset
new_test = dict()
for tr_id in test.iterkeys():
    traj = test[tr_id]
    matched = match_res[tr_id]
    new_points = []
    for idx, point in enumerate(traj):
        rid = matched[point[0]][3]
        obsv = sag.get_obsv(point)
        state = rid
        if B.has_key(obsv) and B[obsv].has_key(state):
            new_points.append(point)
    new_test[tr_id] = new_points
    if len(traj)!=len(new_points):
        print 'TrajID=%d Before=%d End=%d' % (tr_id, len(traj), len(new_test[tr_id]))

In [ ]:
path_pred = dict()
statistic = []
for tr_id, traj in new_test.iteritems():
#     if len(traj) > 100:
#         continue
    sl = set([x[3] for x in match_res[tr_id]])
    try:
        sl, max_prob = hmm.viterbi(A, B, traj)
    except:
        print 'ERR=', tr_id
        continue
#     prob = sag.backforward(A, B, new_test[tr_id], match_res[tr_id], k)
    statistic.append((tr_id, max_prob))
    path_pred[tr_id] = sl
    print 'TrajID=', tr_id

In [ ]:
def evaluate_prep(road_network):
    _, _, city_edgeGeometry = mmr.build_road_network(road_network)
    for rid in city_edgeGeometry.iterkeys():
        points = []
        for lng, lat in list(city_edgeGeometry[rid][1].coords):
            x, y, _, _ = utm.from_latlon(lat, lng)
            points.append((x, y))
        city_edgeGeometry[rid][0] = LineString(points)
    return city_edgeGeometry

edges = evaluate_prep('./data/Shanghai/SH_map/')
path_eval = hmm.evaluate(new_test, path_pred, edges)

In [ ]:
np.median(path_eval.values())

# (HiddenState: Cell)[V3]

In [ ]:
uni_hmm = reload(uni_hmm)
bounding_box = (328780, 330420, 3462200, 3463510)
side = 40
sag = uni_hmm.Sag(side, bounding_box)
A, Cells = sag.get_trans_mat(dbs)
B, D = sag.get_obsv_mat(train)
summary = []
for cid in D.iterkeys():
    to_time = sum(times for obsv, times in D[cid].iteritems())
    to_obsv = len(D[cid])
    summary.append((cid, to_obsv, to_time))
#     print 'Cell[%d] has %d obsv, with %d times...' % (cid, to_obsv, to_time)
df_freq = pd.DataFrame(summary, columns=['CellID', 'Obsv', 'Points'])
df_freq.describe()

In [ ]:
uni_cells = sag.get_cells(Cells)

In [ ]:
plot = reload(plot)
axis = (328780, 330420, 3462200, 3463510)
fig_width = 20
plt.figure(figsize=(fig_width, fig_width*(axis[3]-axis[2])/(axis[1]-axis[0])))
ca = plt.gca()
plot.draw_cells(uni_cells, ca, axis, debug=False)
plt.axis(axis)
plt.title('Uniform cell index with side=' + str(side))
plt.savefig('./display/fig/uni_cell_side=%d.png' % side)
plt.close('all')

In [ ]:
plot = reload(plot)
plot.draw_obsv_barplot(D)

In [ ]:
uni_pred = dict()
for tr_id, traj in test.iteritems():
    if tr_id == 0:
        continue
    sl = sag.viterbi(A, B, traj)
    sag.backforward(A, B, traj)
    uni_pred[tr_id] = sl

In [ ]:
uni_prec, uni_prec_dict = sag.evaluate(test, uni_pred)

In [ ]:
tr_id = 1
df_test = pd.DataFrame(sag.cid_test[tr_id], columns=['tx', 'ty'])
df_pred = pd.DataFrame(sag.cid_pred[tr_id], columns=['px', 'py'])
df_combined = pd.concat([df_test, df_pred], axis=1)
df_combined[80:]

In [ ]:
plot = reload(plot)
axis = (328780, 330420, 3462200, 3463510)
fig_width = 20
for tr_id in uni_pred.iterkeys():
    if tr_id != 7:
        continue
    plt.figure(figsize=(fig_width, fig_width*(axis[3]-axis[2])/(axis[1]-axis[0])))
    ca = plt.gca()
    plot.draw_cells(uni_cells, ca, axis, debug=False)
    plot.draw_traj_on_cells(sag.utm_test[tr_id], ca, axis, 'b')
    plot.draw_traj_on_cells(sag.utm_pred[tr_id], ca, axis, 'g')
    plt.axis(axis)
    plt.title('Uniform cell index with side=' + str(side))
    plt.show()

In [ ]:
def plot_median(precision, xlabel, ylabel, title):
    count = len(precision)
    X = sorted(precision)
    Y = np.cumsum([1.0/count] * count)
    plt.plot(X, Y)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

In [ ]:
np.median(uni_prec)

In [ ]:
plot_median(uni_prec, 'deviation', 'CDF', 'Uniform Cell Localization(ID1)')

In [ ]:
# clean test dataset
new_test = dict()
for tr_id in test.iterkeys():
    traj = test[tr_id]
    matched = match_res[tr_id]
    new_points = []
    for idx, point in enumerate(traj):
        piece_match = matched[point[0]]
        lat, lng = piece_match[4], piece_match[5]
        x, y, _, _ = utm.from_latlon(lat, lng)
        cid = sag.utm2cell(x, y)
        obsv = sag.get_obsv(point)
        state = cid
        if B[obsv].has_key(state):
            new_points.append(point)
    new_test[tr_id] = new_points
    if len(traj)!=len(new_points):
        print 'TrajID=%d Before=%d End=%d' % (tr_id, len(traj), len(new_test[tr_id]))

In [ ]:
for tr_id in new_test.iterkeys():
    traj = new_test[tr_id]
    matched = match_res[tr_id]
    new_points = []
    for idx, point in enumerate(traj):
        piece_match = matched[point[0]]
        lat, lng = piece_match[4], piece_match[5]
        x, y, _, _ = utm.from_latlon(lat, lng)
        cid = sag.utm2cell(x, y)
        obsv = sag.get_obsv(point)
        state = cid
        if not B[obsv].has_key(state):
            print tr_id, obsv, state

In [ ]:
points = []
for tr_id, traj in gps3.iteritems():
    for point in traj:
        points.append((point[0], point[1]))
display_osm.df_to_html(pd.DataFrame(points, columns=['Latitude', 'Longitude']), './display/gps3.html')
points = []
for tr_id, traj in train.iteritems():
    for point in traj:
        points.append((point[1][0], point[1][1]))
display_osm.df_to_html(pd.DataFrame(points, columns=['Latitude', 'Longitude']), './display/train.html')
points = []
for tr_id, traj in test.iteritems():
    for point in traj:
        points.append((point[1][0], point[1][1]))
display_osm.df_to_html(pd.DataFrame(points, columns=['Latitude', 'Longitude']), './display/test.html')

In [ ]:
for tr_id, traj in train.iteritems():
    points = []
    for point in traj:
        points.append((point[1][0], point[1][1]))
    display_osm.df_to_html(pd.DataFrame(points, columns=['Latitude', 'Longitude']), './display/train/%d.html' % tr_id)
for tr_id, traj in test.iteritems():
    points = []
    for point in traj:
        points.append((point[1][0], point[1][1]))
    display_osm.df_to_html(pd.DataFrame(points, columns=['Latitude', 'Longitude']), './display/test/%d.html' % tr_id)

In [ ]:
tr_id = 19
traj = new_test[tr_id]
speeds = [point[1][-2] for point in traj]
k = math.floor(np.std(speeds))
print 'k=' + str(k)
sag.backforward(A, B, traj, match_res[tr_id], k, debug=True)
f, p = [], defaultdict(dict)
h = dict()
self = sag
obsv = self.get_obsv(traj[0])
for rid, prob in B[obsv].iteritems():
    h[rid] = prob
f.append(h)
for idx in range(1, len(traj)):
    obsv = self.get_obsv(traj[idx])
    speed, timestp = traj[idx][1][-2:]
    h = dict()
    _obsv = self.get_obsv(traj[idx-1])
    _speed, _timestp = traj[idx-1][1][-2:]
    max_dist = max(_speed, speed) * (timestp - _timestp) + 1.0 * self.side
    min_dist = min(_speed, speed) * (timestp - _timestp) - 1.0 * self.side
    for rid, prob in B[obsv].iteritems():
        max_p = 0
        x1, y1 = self.cell2index(rid)
        for _rid in B[_obsv].iterkeys():
            if not f[idx-1].has_key(_rid):
                continue
            x2, y2 = self.cell2index(_rid)
            avg_act_dist = (math.sqrt((x2-x1)**2 + (y2-y1)**2)) * self.side
            t1 = t2 = t3 = 0
            t1 = f[idx-1][_rid]
            t3 = B[obsv][rid]
            if not (min_dist - self.side * k <= avg_act_dist <= max_dist + self.side * k):
                continue
            t2 = self.transfer(_rid, rid)
            if avg_act_dist < min_dist:
                t2 *= math.e ** (-(avg_act_dist - min_dist)**2/(min_dist**2)/2)
            if avg_act_dist > max_dist:
                t2 *= math.e ** (-(avg_act_dist - max_dist)**2/(max_dist**2)/2)
            alt_p = t1 + t2 * t3
            if alt_p > max_p:
                max_p = alt_p
                p[idx][rid] = _rid
            h[rid] = max_p
    f.append(h)
    print idx
sl = []
max_prob = max(f[-1].itervalues())
last_s = max(f[-1].iterkeys(), key=lambda k: f[-1][k])
for idx in range(len(traj)-1, 0, -1):
    sl.append(last_s)
    last_s = p[idx][last_s]
sl.append(last_s)
sl.reverse()

In [ ]:
_x, _y = new_test[tr_id][0][1][:2]
for idx, point in enumerate(new_test[tr_id]):
    print idx, point[1]
    x, y = point[1][:2]
    if idx >= 1:
        print math.sqrt((x - _x)**2+(y - _y)**2)
    _x, _y = x, y

# (HiddenState:K-D Cell)[V2]

In [ ]:
for i, point in enumerate(obsv):
    lat, lng, _, _ = point
    x, y, _, _ = utm.from_latlon(lat, lng)
    obsv[i] = (x, y)
df_obsv = pd.DataFrame(obsv, columns=['x', 'y'])
min(df_obsv['x']), max(df_obsv['x']), min(df_obsv['y']), max(df_obsv['y'])

In [ ]:
r_idx, sp_dict, cnt = kd_hmm.kd_split(obsv, axis=0, out_box=[[328780, 330420], [3462200, 3463510]])
print 'dividing', cnt, ' times'

In [ ]:
plot = reload(plot)
axis = (328780, 330420, 3462200, 3463510)
fig_width = 20
plt.figure(figsize=(fig_width, fig_width*(axis[3]-axis[2])/(axis[1]-axis[0])))
ca = plt.gca()
plot.draw_kd_tree(sp_dict, ca, axis, debug=False)
plt.axis(axis)
plt.title('K-D tree spatial index')
# plt.show()
plt.savefig('./display/fig/kdtree_side=80.png')
plt.close('all')

## random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
def re_organize(db):
    X, y = [], []
    for tr_id, traj in db.iteritems():
        piece_match = match_res[tr_id]
        for old_idx, point in traj:
            obsv = point[2:-2]
            rid = piece_match[old_idx][3]
            X.append(obsv)
            y.append(rid)
    return X, y
X_train, Y_train = re_organize(train)
X_test, Y_test = re_organize(new_test)

clf = RandomForestClassifier(n_estimators=200)
clf.fit(X_train, Y_train)
clf.feature_importances_

Y_predict = clf.predict_proba(X_test)

cnt = 0
for i, piece in enumerate(Y_predict):
    k = min(5, len(set(piece)) - 1)
#     k = 1
    idxes = sorted(range(len(piece)), key=lambda x: piece[x])[-k:]
    rids = [clf.classes_[idx] for idx in idxes]
    if not Y_test[i] in rids:
        cnt += 1
cnt, len(Y_test)